In [1]:
import cv2
import torch
import torchvision.ops.boxes as bops
import os

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from tqdm.notebook import tqdm

In [4]:
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [5]:
path = '../Satellite-Aerial-Image-Retrieval/output/'

images_path = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.jpeg' in file and "150m" not in file: 
            images_path.append(os.path.join(r, file))
images_path.sort()

In [6]:
# model_name = "yolov5s_576_d15_t0.05_s0.4_bs3_wd0.0.05_lr0.001_newdat"

In [7]:
model_name = "yolov5s_576_d15_t0.35_s0.3_bs3_wd0.0.05_lr0.001_newdat"

In [8]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path=f'weights/{model_name}.pt') 

Using cache found in /home/hue/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-9-11 torch 1.9.0+cu102 CUDA:0 (NVIDIA GeForce MX250, 2002.625MB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
Adding AutoShape... 


In [9]:
fmodel_name = "last"

In [10]:
fmodel = torch.hub.load('ultralytics/yolov5', 'custom', path=f'weights/{fmodel_name}.pt')  

Using cache found in /home/hue/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-9-11 torch 1.9.0+cu102 CUDA:0 (NVIDIA GeForce MX250, 2002.625MB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
Adding AutoShape... 


In [11]:
def plot_bbox(results_xyxy, image, confidence_threshold):
    output_image  = image.copy()
    for result in results_xyxy:
        xmin = int(result[0].item())
        ymin = int(result[1].item()) 
        xmax = int(result[2].item())
        ymax = int(result[3].item())
        confidence = float(result[4].item())
        if confidence >= confidence_threshold:
            output_image = cv2.rectangle(output_image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 3)
            # Using cv2.putText() method
            output_image = cv2.putText(output_image, f'{confidence*100:0.2f}', (xmin+10 ,ymin+30), cv2.FONT_HERSHEY_SIMPLEX, 
                               0.75, (255, 0, 0), 2, cv2.LINE_AA)
    return output_image

In [12]:
model.conf = 0.25  # confidence threshold (0-1)
model.iou = 0.0  # NMS IoU threshold (0-1)

In [14]:
from itertools import combinations

def remove_duplicates(detected_areas, iou_threshold):
    areas = detected_areas
    coordinates =  list(combinations([coordinate for _, coordinate in detected_areas], 2))
    to_removes = []
    for box1, box2 in coordinates:
        iou = bops.box_iou(torch.tensor([box1], dtype=torch.float), torch.tensor([box2], dtype=torch.float))
#         print(iou)
        if iou > iou_threshold:
#             print(box1, box2)
            to_removes.append(box2)
    for to_remove in to_removes:
        areas = [[image, cordinate] for image, cordinate in areas if cordinate != to_remove]
    return areas

In [15]:
bs = 50
for path in tqdm(images_path[:]):
    name = os.path.basename(path)[:-5]
#     output_path_raw = f"output/{model_name}/{name}"
#     os.makedirs(output_path_raw, exist_ok=True)
    output_path_annotated = f"output/{model_name}/{name}_annotated"
    os.makedirs(output_path_annotated, exist_ok=True)
    bgr_image = cv2.imread(path)
    input_max_x, input_max_y, _ = bgr_image.shape
    image_rgb = cv2.cvtColor(bgr_image,cv2.COLOR_BGR2RGB)  # OpenCV image (BGR to RGB)
    tiles = [[image, (x,y)] for x,y, image in sliding_window(image_rgb, 750, (1000, 1000)) if image.shape[0]==image.shape[1]]
    
    detected_areas = []
    for i in tqdm(range(0, len(tiles), bs), leave=False):
        chunk = tiles[i:i + bs]
        images = [image for image, _  in chunk]
        results = model(images, size=576)
#         results.save()
        results.render()
        for idx, result in enumerate(results.pred):
            if len(result) != 0:
                x,y = chunk[idx][1]
                for result in results.xyxy[idx]:
                    xmin = int(result[0].item())
                    ymin = int(result[1].item()) 
                    xmax = int(result[2].item())
                    ymax = int(result[3].item())  
                    
                    yb = y+(ymin+ymax)//2-450
                    if yb<0:
                        yb = 0
                        
                    yt = y+(ymin+ymax)//2+550
                    if yt > input_max_y:
                        yt = input_max_y
                        
                    xl = x+(xmin+xmax)//2-480
                    if xl < 0:
                        xl = 0
                        
                    xr = x+(xmin+xmax)//2+520
                    if xr > input_max_x:
                        xr = input_max_x
                        
                    new_image = image_rgb[yb:yt,xl:xr]
                    detected_areas.append([new_image, (xl, yb,xr, yt)])
#                 cv2.imwrite(f"{output_path_raw}/{name}_{i}_{idx}.png",  cv2.cvtColor(chunk[idx][0], cv2.COLOR_RGB2BGR))
                cv2.imwrite(f"{output_path_annotated}/{name}_{i}_{idx}.png",  cv2.cvtColor(images[idx], cv2.COLOR_RGB2BGR))
    
#     output_path_final = f"output/{model_name}/{name}_scale_invariant"
#     os.makedirs(output_path_final, exist_ok=True)
    detected_areas = remove_duplicates(detected_areas, 0.50)
    confidence_threshold = 0.5
    
#     for i in tqdm(range(0, len(detected_areas), bs), leave=False):
#         chunk = detected_areas[i:i + bs]
#         images = [image for image, _  in chunk]
#         nresults = fmodel(images, size=576)
        
#         for idx in range(len(nresults)):
#             results_xyxy = nresults.xyxy[idx] 
#             if len(results_xyxy) == 0:
#                 continue
#             confidence = [float(result_xyxy[4].item()) for result_xyxy in results_xyxy]
#             if max(confidence) >= confidence_threshold:
#                 image = images[idx]
#                 cv2.imwrite(f"{output_path_final}/{name}_{idx}.png",
#                             cv2.cvtColor(plot_bbox(results_xyxy, image, confidence_threshold), cv2.COLOR_RGB2BGR))

    output_path_final = f"output/{model_name}/{name}_filtered"
    os.makedirs(output_path_final, exist_ok=True)

    
    for i in tqdm(range(0, len(detected_areas), bs), leave=False):
        chunk = detected_areas[i:i + bs]
        images = [image for image, _  in chunk]
        nresults = model(images, size=576)
        
        for idx in range(len(nresults)):
            results_xyxy = nresults.xyxy[idx] 
            if len(results_xyxy) == 0:
                continue
            confidence = [float(result_xyxy[4].item()) for result_xyxy in results_xyxy]
            if max(confidence) >= confidence_threshold:
                image = images[idx]
                image = cv2.cvtColor(plot_bbox(results_xyxy, image, confidence_threshold), cv2.COLOR_RGB2BGR)
                image = cv2.resize(image,(600,600), interpolation = cv2.INTER_AREA)
                 
                cv2.imwrite(f"{output_path_final}/{name}_{idx}.png",
                            image,
                            [int(cv2.IMWRITE_PNG_COMPRESSION),5])


In [ ]:
# https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation

In [ ]:
len(detected_areas)

In [ ]:
detected_areas = remove_duplicates(detected_areas, 0.50)

In [ ]:
# plt.figure(figsize=(10, 10))
# plt.imshow(detected_areas[4][0])
# plt.show()

In [ ]:
len(detected_areas)

In [ ]:
detected_areas[:][1]